In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('模型数据.xlsx')
data.head(2)

,crash,time of week,weather,volume,temperature,daytime,season
0,1,1,cloudy,0,0,afternoon,winter
1,1,1,cloudy,0,0,afternoon,winter


# 贝叶斯网络构建

In [3]:
from pgmpy.models import BayesianNetwork
model = BayesianNetwork([('time of week', 'temperature'),('time of week','volume'),
                         ('daytime','temperature'),('daytime','volume'),
                         ('season','temperature'),('season','volume'),('season','weather'),
                         ('weather','temperature'),('weather','volume'),('weather','crash'),
                         ('temperature','crash'),
                         ('volume','crash')])

# 模型评估

In [4]:
from pgmpy.metrics import structure_score
k2 = structure_score(model, data, scoring_method="k2") 
bdeu = structure_score(model, data, scoring_method="bdeu") 
bds  = structure_score(model, data, scoring_method="bds") 
bic = structure_score(model, data, scoring_method="bic")
print("K2:", k2)
print("BDeu:", bdeu)
print("BDs", bds)
print("BIC:", bic)

K2: -238913.00812345115
BDeu: -239081.36505176625
BDs -240110.7529786899
BIC: -243757.27993152925


In [8]:
model1 = BayesianNetwork([('time of week','volume'),
                          ('volume','crash')])
data = data[['time of week','volume','crash']]
k2 = structure_score(model1, data, scoring_method="k2") 
bdeu = structure_score(model1, data, scoring_method="bdeu") 
bds  = structure_score(model1, data, scoring_method="bds") 
bic = structure_score(model1, data, scoring_method="bic")
print("K2:", k2)
print("BDeu:", bdeu)
print("BDs", bds)
print("BIC:", bic)

K2: -66567.44279271574
BDeu: -66572.1302607096
BDs -66575.5959966124
BIC: -66572.84078519492


# 网络参数估计

In [4]:
from pgmpy.estimators import BayesianEstimator, MaximumLikelihoodEstimator
estimator = MaximumLikelihoodEstimator(model, data)

In [5]:
cpd = estimator.estimate_cpd('crash')
print(cpd.variables)
print(cpd.cardinality)
pd.DataFrame(cpd.get_values())

['crash', 'temperature', 'volume', 'weather']
[2 3 3 5]


,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,0.975464,0.983778,0.969646,0.925134,0.980999,0.941794,0.972851,0.93662,0.813665,0.951542,...,0.965665,0.965714,0.966862,0.5,0.922807,0.96308,0.97561,0.970588,0.5,0.982609
1,0.024536,0.016222,0.030354,0.074866,0.019001,0.058206,0.027149,0.06338,0.186335,0.048458,...,0.034335,0.034286,0.033138,0.5,0.077193,0.03692,0.02439,0.029412,0.5,0.017391


In [6]:
cpd_matric = pd.DataFrame(cpd.get_values())
column_num = cpd_matric.shape[1]

# 事故的CPD

In [7]:
CPD = pd.DataFrame()

In [8]:
#crash:1 tem
list0 = np.array([])
list1 = np.array([])
list2 = np.array([])
large_unit = 0
unit = int(column_num/cpd.cardinality[1])
list0 = np.append(list0,np.linspace(int(large_unit),int(large_unit + unit)-1,int(unit))).astype(int)
list1 = np.append(list1,np.linspace(int(large_unit + unit),int(large_unit + 2*unit)-1,int(unit))).astype(int)
list2 = np.append(list2,np.linspace(int(large_unit + 2*unit),int(large_unit + 3*unit)-1,int(unit))).astype(int)

t0 = np.array([])
for i in list0:
    t0 = np.append(t0,cpd_matric[i][1])

t1 = np.array([])
for i in list1:
    t1 = np.append(t1,cpd_matric[i][1])

t2 = np.array([])
for i in list2:
    t2 = np.append(t2,cpd_matric[i][1])

CPD['crash_1_temperature_0'] = pd.DataFrame(t0)
CPD['crash_1_temperature_1'] = pd.DataFrame(t1)
CPD['crash_1_temperature_2'] = pd.DataFrame(t2)

In [9]:
#crash:1 low_tem
list0 = np.array([])
list1 = np.array([])
list2 = np.array([])
large_unit = int(column_num/(cpd.cardinality[1]))
unit = int(column_num/(cpd.cardinality[1]*cpd.cardinality[2]))
for i in range(cpd.cardinality[1]):
    list0 = np.append(list0,np.linspace(int(i*large_unit),int(i*large_unit+ unit)-1,int(unit))).astype(int)
    list1 = np.append(list1,np.linspace(int(i*large_unit + unit),int(i*large_unit+ 2*unit)-1,int(unit))).astype(int)
    list2 = np.append(list2,np.linspace(int(i*large_unit + 2*unit),int(i*large_unit+ 3*unit)-1,int(unit))).astype(int)

t0 = np.array([])
for i in list0:
    t0 = np.append(t0,cpd_matric[i][1])

t1 = np.array([])
for i in list1:
    t1 = np.append(t1,cpd_matric[i][1])

t2 = np.array([])
for i in list2:
    t2 = np.append(t2,cpd_matric[i][1])

CPD['crash_1_volume_0'] = pd.DataFrame(t0)
CPD['crash_1_volume_1'] = pd.DataFrame(t1)
CPD['crash_1_volume_2'] = pd.DataFrame(t2)

In [10]:
#crash:1 month
list0 = np.array([])
list1 = np.array([])
list2 = np.array([])
list3 = np.array([])
list4 = np.array([])
t0 = np.array([])
t1 = np.array([])
t2 = np.array([])
t3 = np.array([])
t4 = np.array([])

large_unit = int(column_num/(cpd.cardinality[1]*cpd.cardinality[2]))
unit = int(column_num/(cpd.cardinality[1]*cpd.cardinality[2]*cpd.cardinality[3]))
for i in range(cpd.cardinality[1]*cpd.cardinality[2]):
    list0 = np.append(list0,np.linspace(int(i*large_unit + 0*unit),int(i*large_unit+ 1*unit)-1,int(unit))).astype(int)
    list1 = np.append(list1,np.linspace(int(i*large_unit + 1*unit),int(i*large_unit+ 2*unit)-1,int(unit))).astype(int)
    list2 = np.append(list2,np.linspace(int(i*large_unit + 2*unit),int(i*large_unit+ 3*unit)-1,int(unit))).astype(int)
    list3 = np.append(list3,np.linspace(int(i*large_unit + 3*unit),int(i*large_unit+ 4*unit)-1,int(unit))).astype(int)
    list4 = np.append(list4,np.linspace(int(i*large_unit + 4*unit),int(i*large_unit+ 5*unit)-1,int(unit))).astype(int)

for i in list0:
    t0 = np.append(t0,cpd_matric[i][1])

for i in list1:
    t1 = np.append(t1,cpd_matric[i][1])

for i in list2:
    t2 = np.append(t2,cpd_matric[i][1])

for i in list3:
    t3 = np.append(t3,cpd_matric[i][1])

for i in list4:
    t4 = np.append(t4,cpd_matric[i][1])


CPD['crash_1_weather_0'] = pd.DataFrame(t0)
CPD['crash_1_weather_1'] = pd.DataFrame(t1)
CPD['crash_1_weather_2'] = pd.DataFrame(t2)
CPD['crash_1_weather_3'] = pd.DataFrame(t3)
CPD['crash_1_weather_4'] = pd.DataFrame(t4)

In [11]:
print(CPD.shape)
CPD.head(2)

(15, 11)


,crash_1_temperature_0,crash_1_temperature_1,crash_1_temperature_2,crash_1_volume_0,crash_1_volume_1,crash_1_volume_2,crash_1_weather_0,crash_1_weather_1,crash_1_weather_2,crash_1_weather_3,crash_1_weather_4
0,0.024536,0.012014,0.016139,0.024536,0.058206,0.020367,0.024536,0.016222,0.030354,0.074866,0.019001
1,0.016222,0.024659,0.029685,0.016222,0.027149,0.010870,0.058206,0.027149,0.063380,0.186335,0.048458


In [12]:
CPD.to_excel('CPD_crash_1.xlsx',index=None)

# 贝叶斯网络推断

In [14]:
model.fit(data)

In [15]:
from pgmpy.inference import VariableElimination
inference = VariableElimination(model)

In [16]:
inference = inference.query(['time of week','daytime','weather','season','temperature','volume','crash'],joint=True)

In [34]:
print(inference)

+-----------------+--------------------+-----------------+----------------+----------------+-----------+----------+---------------------------------------------------------------------+
| time of week    | daytime            | weather         | season         | temperature    | volume    | crash    |   phi(time of week,daytime,weather,season,temperature,volume,crash) |
+=================+====================+=================+================+================+===========+==========+=====================================================================+
| time of week(0) | daytime(afternoon) | weather(cloudy) | season(autumn) | temperature(0) | volume(0) | crash(0) |                                                              0.0004 |
+-----------------+--------------------+-----------------+----------------+----------------+-----------+----------+---------------------------------------------------------------------+
| time of week(0) | daytime(afternoon) | weather(cloudy) | season(autu

In [17]:
inference.variables

['time of week',
 'daytime',
 'weather',
 'season',
 'temperature',
 'volume',
 'crash']

In [18]:
inference.cardinality

array([3, 6, 5, 4, 3, 3, 2])

In [19]:
infer_values = inference.values
print(type(infer_values))
print(infer_values.shape)
infer_values = infer_values.reshape(3*6*5*4*3*3*2,1)
#pd.DataFrame(infer_values).to_excel('inference_data.xlsx',index=None)

<class 'numpy.ndarray'>
(3, 6, 5, 4, 3, 3, 2)


In [20]:
infer_values = pd.DataFrame(infer_values)
infer_values.columns = ['p']

In [21]:
crash = [int(np.floor(i%2)) for i in range(len(infer_values))]
infer_values = pd.concat([infer_values,pd.DataFrame(crash,columns=['crash'])],axis=1)

In [22]:
volume = [int(np.floor(i%(2*3))) for i in range(len(infer_values))]
volume = [int(np.floor(i/2)) for i in volume]
infer_values = pd.concat([infer_values,pd.DataFrame(volume,columns=['volume'])],axis=1)

In [23]:
temperature = [int(np.floor(i%(2*3*3))) for i in range(len(infer_values))]
temperature = [int(np.floor(i/(2*3))) for i in temperature]
infer_values = pd.concat([infer_values,pd.DataFrame(temperature,columns=['temperature'])],axis=1)

In [24]:
season = [int(np.floor(i%(2*3*3*4))) for i in range(len(infer_values))]
season = [int(np.floor(i/(2*3*3))) for i in season]
infer_values = pd.concat([infer_values,pd.DataFrame(season,columns=['season'])],axis=1)

In [25]:
weather = [int(np.floor(i%(2*3*3*4*5))) for i in range(len(infer_values))]
weather = [int(np.floor(i/(2*3*3*4))) for i in weather]
infer_values = pd.concat([infer_values,pd.DataFrame(weather,columns=['weather'])],axis=1)

In [26]:
daytime = [int(np.floor(i%(2*3*3*4*5*6))) for i in range(len(infer_values))]
daytime = [int(np.floor(i/(2*3*3*4*5))) for i in daytime]
infer_values = pd.concat([infer_values,pd.DataFrame(daytime,columns=['daytime'])],axis=1)

In [27]:
time_of_week = [int(np.floor(i%(2*3*3*4*5*6*3))) for i in range(len(infer_values))]
time_of_week = [int(np.floor(i/(2*3*3*4*5*6))) for i in time_of_week]
infer_values = pd.concat([infer_values,pd.DataFrame(time_of_week,columns=['time of week'])],axis=1)

In [28]:
infer_values

,p,crash,volume,temperature,season,weather,daytime,time of week
0,0.000403,0,0,0,0,0,0,0
1,0.000010,1,0,0,0,0,0,0
2,0.000222,0,1,0,0,0,0,0
3,0.000014,1,1,0,0,0,0,0
4,0.000197,0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...
6475,0.000000,1,0,2,3,4,5,2
6476,0.000000,0,1,2,3,4,5,2
6477,0.000000,1,1,2,3,4,5,2
6478,0.000000,0,2,2,3,4,5,2


In [29]:
def cseason(x):
    t = 0
    if (x-0)<0.1:
        t = 'autumn'
    elif (x-1)<0.1:
        t = 'spring'
    elif (x-2)<0.1:
        t = 'summer'
    elif (x-3)<0.1:
        t = 'winter'
    return t
infer_values['season'] = infer_values.apply(lambda infer_values:cseason(infer_values['season']),axis=1)

In [30]:
def cweather(x):
    t = 0
    if (x-0)<0.1:
        t = 'cloudy'
    elif (x-1)<0.1:
        t = 'foggy'
    elif (x-2)<0.1:
        t = 'rainy'
    elif (x-3)<0.1:
        t = 'snowy'
    elif (x-4)<0.1:
        t = 'sunny'
    return t
infer_values['weather'] = infer_values.apply(lambda infer_values:cweather(infer_values['weather']),axis=1)

In [31]:
def cdaytime(x):
    t = 0
    if (x-0)<0.1:
        t = 'afternoon'
    elif (x-1)<0.1:
        t = 'dawn'
    elif (x-2)<0.1:
        t = 'evening'
    elif (x-3)<0.1:
        t = 'morning'
    elif (x-4)<0.1:
        t = 'night'
    elif (x-5)<0.1:
        t = 'noon'
    return t
infer_values['daytime'] = infer_values.apply(lambda infer_values:cdaytime(infer_values['daytime']),axis=1)

In [32]:
infer_values

,p,crash,volume,temperature,season,weather,daytime,time of week
0,0.000403,0,0,0,autumn,cloudy,afternoon,0
1,0.000010,1,0,0,autumn,cloudy,afternoon,0
2,0.000222,0,1,0,autumn,cloudy,afternoon,0
3,0.000014,1,1,0,autumn,cloudy,afternoon,0
4,0.000197,0,2,0,autumn,cloudy,afternoon,0
...,...,...,...,...,...,...,...,...
6475,0.000000,1,0,2,winter,sunny,noon,2
6476,0.000000,0,1,2,winter,sunny,noon,2
6477,0.000000,1,1,2,winter,sunny,noon,2
6478,0.000000,0,2,2,winter,sunny,noon,2


In [33]:
infer_values.to_excel('推断数据.xlsx',index=None)